In [1]:
from src.config.data_loader_config import DATA_LOADER_CONFIG, OPTIMIZER_CONFIG
from src.data_loader.loader import Dataloader
from src.model.model import Model, Models, LossFunctions
from src.trainer.predict import save_result
import src.callback as callback
import pytorch_lightning as pl
import os

# Parameters 설정
batch_size = DATA_LOADER_CONFIG['batch_size']
shuffle = DATA_LOADER_CONFIG['shuffle']
learning_rate = OPTIMIZER_CONFIG['learning_rate']
max_epoch = OPTIMIZER_CONFIG['max_epoch']
os.environ["TOKENIZERS_PARALLELISM"] = "false"
num_workers: int = DATA_LOADER_CONFIG.get('num_workers', 4)

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 모델 선언
# select Model -> roberta_base, roberta_small, roberta_large, electra_base, electra_base_v3
model_name = 'electra_base_v3'
model = Model(Models.electra_base_v3, learning_rate, LossFunctions.hu_loss)

# Dataloader 선언
dataloader = Dataloader(batch_size, shuffle, Models.electra_base_v3, check_aug=True)

# callback 정의
epoch_print_callback = callback.EpochPrintCallback()
checkpoint_callback = callback.ModelCheckpoint(model_name=model_name)
early_stopping = callback.EarlyStopping()
lr_monitor = callback.LearningRateMonitor()

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 학습

In [4]:
trainer = pl.Trainer(accelerator='gpu', devices='auto', max_epochs=max_epoch, callbacks=[lr_monitor, epoch_print_callback,checkpoint_callback, early_stopping], precision='16-mixed')
trainer.fit(model=model, datamodule=dataloader)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
/data/ephemeral/home/level1-semantictextsimilarity-nlp-07/src/preprocessing/preprocessor.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_zero_labels[['sentence_1', 'sentence_2']] = non_zero_labels[['sentence_2', 'sentence_1

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/loss.py:993: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.huber_loss(input, target, reduction=self.reduction, delta=self.delta)
/opt/conda/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: The variance of predictions or target is close to zero. This can cause instability in Pearson correlationcoefficient, leading to wrong results. Consider re-scaling the input if possible or computing using alarger dtype (currently using torch.float16).
  warnings.warn(*args, **kwargs)  # noqa: B028


Epoch 0: 100%|██████████| 1033/1033 [00:57<00:00, 17.84it/s, v_num=8]

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/loss.py:993: UserWarning: Using a target size (torch.Size([15])) that is different to the input size (torch.Size([15, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.huber_loss(input, target, reduction=self.reduction, delta=self.delta)


...

Epoch 9, global step 10330: 'val_loss' was not in top 1


Epoch 10: 100%|██████████| 1033/1033 [00:59<00:00, 17.51it/s, v_num=8]Epoch 10 ended
Validation Loss: 0.8686
Training Loss: 0.9667
Validation Pearson Correlation: 0.8404
----------------------------------------
Epoch 10: 100%|██████████| 1033/1033 [00:59<00:00, 17.51it/s, v_num=8]

Epoch 10, global step 11363: 'val_loss' was not in top 1


Epoch 11:  52%|█████▏    | 536/1033 [00:30<00:27, 17.78it/s, v_num=8] 

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [ ]:
import torch
torch.cuda.empty_cache()

# 추론


In [ ]:
checkpoint_callback.best_model_path

''

In [ ]:
# 가장 좋은 모델 불러오기
best_model_path = checkpoint_callback.best_model_path
model = Model.load_from_checkpoint(best_model_path, loss_func=LossFunctions.hu_loss)
trainer.test(model=model, datamodule=dataloader)
# 추론
predictions = trainer.predict(model=model, datamodule=dataloader)

# 결과 저장
save_result(predictions, model_name, max_epoch)

NameError: name 'checkpoint_callback' is not defined